## Importing and Exploring the datasets
Steps will be:
- Import the data
- Check out the data
- Check the info
- Describe the data
- Check the shape

In [105]:
import sqlite3
import pandas as pd

In [117]:
# Imports the data
bom_raw = pd.read_csv("../data/bom.movie_gross.csv.gz")
conn = sqlite3.connect("../data/im.db")
rt_movie_info_raw = pd.read_csv("../data/rt.movie_info.tsv.gz", delimiter="\t")
rt_reviews_raw = pd.read_csv("../data/rt.reviews.tsv.gz", delimiter="\t", encoding='cp1252')
tmdb_movies_raw = pd.read_csv("../data/tmdb.movies.csv.gz")
tn_movie_budgets_raw = pd.read_csv("../data/tn.movie_budgets.csv.gz")

In [107]:
# rt_review_data_raw initially doesn't load due to an encoding error. Here we detect the encoding type before inserting it into the codeblock above
with open("../data/rt.reviews.tsv.gz") as rt_reviews:
    print(rt_reviews)

<_io.TextIOWrapper name='../data/rt.reviews.tsv.gz' mode='r' encoding='cp1252'>


***Checking out the Data***

In [108]:
# Check out the data
bom_raw.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [109]:
imdb_raw = pd.read_sql(
    """
    SELECT *
    FROM sqlite_master
    """
, conn
)
imdb_raw

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [118]:
# Get a list of all tables in the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
imdb_tables = cursor.fetchall()

# Loop through each table and display its head using pandas
for table_name in imdb_tables:
    df = pd.read_sql_query(f"SELECT * from {table_name[0]}", conn)
    print(f"Table: {table_name[0]}")
    print(df.head(2),"\n")

Table: movie_basics
    movie_id                    primary_title   original_title  start_year  \
0  tt0063540                        Sunghursh        Sunghursh        2013   
1  tt0066787  One Day Before the Rainy Season  Ashad Ka Ek Din        2019   

   runtime_minutes              genres  
0            175.0  Action,Crime,Drama  
1            114.0     Biography,Drama   

Table: directors
    movie_id  person_id
0  tt0285252  nm0899854
1  tt0462036  nm1940585 

Table: known_for
   person_id   movie_id
0  nm0061671  tt0837562
1  nm0061671  tt2398241 

Table: movie_akas
    movie_id  ordering              title region language        types  \
0  tt0369610        10      Джурасик свят     BG       bg         None   
1  tt0369610        11  Jurashikku warudo     JP     None  imdbDisplay   

  attributes  is_original_title  
0       None                0.0  
1       None                0.0   

Table: movie_ratings
     movie_id  averagerating  numvotes
0  tt10356526            8.3     

In [111]:
rt_movie_info_raw.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [112]:
rt_reviews_raw.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [113]:
tmdb_movies_raw.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [114]:
tn_movie_budgets_raw.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


**Checking the Info of Each Database**

In [115]:
# Check the information of the data set
bom_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [116]:
# Loop through each table and display its head using pandas
for table_name in imdb_tables:
    df = pd.read_sql_query(f"SELECT * from {table_name[0]}", conn)
    print(f"Table: {table_name[0]}")
    print(df.info())

Table: movie_basics
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB
None
Table: directors
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291174 entries, 0 to 291173
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   291174 non-null  object
 1   person_id  291174 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB
None
Table: known_for
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1638260 entries, 0 to 163825

KeyboardInterrupt: 

In [ ]:
rt_movie_info_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [ ]:
rt_reviews_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [ ]:
tmdb_movies_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [ ]:
tn_movie_budgets_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


**Observations**
- Currency fields will need to be converted to a consistent datatype (ideally float)
- We will have to handle dates
- There are a significant number of nulls

In [ ]:
# Describe the numerical data
print(bom_raw.describe(),"\n")
print(bom_raw.describe(include=object))

       domestic_gross         year
count    3.359000e+03  3387.000000
mean     2.874585e+07  2013.958075
std      6.698250e+07     2.478141
min      1.000000e+02  2010.000000
25%      1.200000e+05  2012.000000
50%      1.400000e+06  2014.000000
75%      2.790000e+07  2016.000000
max      9.367000e+08  2018.000000 

            title studio foreign_gross
count        3387   3382          2037
unique       3386    257          1204
top     Bluebeard    IFC       1200000
freq            2    166            23


In [ ]:
# Loop through each table and display its head using pandas
for table_name in imdb_tables:
    df = pd.read_sql_query(f"SELECT * from {table_name[0]}", conn)
    print(f"Table: {table_name[0]}")
    print(df.describe(),"\n")
    print(df.describe(include=object),"\n")

Table: movie_basics
          start_year  runtime_minutes
count  146144.000000    114405.000000
mean     2014.621798        86.187247
std         2.733583       166.360590
min      2010.000000         1.000000
25%      2012.000000        70.000000
50%      2015.000000        87.000000
75%      2017.000000        99.000000
max      2115.000000     51420.000000 

         movie_id primary_title original_title       genres
count      146144        146144         146123       140736
unique     146144        136071         137773         1085
top     tt0063540          Home         Broken  Documentary
freq            1            24             19        32185 

Table: directors
         movie_id  person_id
count      291174     291174
unique     140417     109253
top     tt4050462  nm6935209
freq         3818        238 

         movie_id  person_id
count      291174     291174
unique     140417     109253
top     tt4050462  nm6935209
freq         3818        238 

Table: known_for
      

In [ ]:
print(rt_movie_info_raw.describe(),"\n")
print(rt_movie_info_raw.describe(include=object))

                id
count  1560.000000
mean   1007.303846
std     579.164527
min       1.000000
25%     504.750000
50%    1007.500000
75%    1503.250000
max    2000.000000 

                                                 synopsis rating  genre  \
count                                                1498   1557   1552   
unique                                               1497      6    299   
top     A group of air crash survivors are stranded in...      R  Drama   
freq                                                    2    521    151   

                director       writer theater_date     dvd_date currency  \
count               1361         1111         1201         1201      340   
unique              1125         1069         1025          717        1   
top     Steven Spielberg  Woody Allen  Jan 1, 1987  Jun 1, 2004        $   
freq                  10            4            8           11      340   

       box_office     runtime              studio  
count         340 

In [ ]:
rt_reviews_raw.describe()

,id,top_critic
count,54432.000000,54432.000000
mean,1045.706882,0.240594
std,586.657046,0.427448
min,3.000000,0.000000
25%,542.000000,0.000000
50%,1083.000000,0.000000
75%,1541.000000,0.000000
max,2000.000000,1.000000


In [ ]:
tmdb_movies_raw.describe()


,Unnamed: 0,id,popularity,vote_average,vote_count
count,26517.00000,26517.000000,26517.000000,26517.000000,26517.000000
mean,13258.00000,295050.153260,3.130912,5.991281,194.224837
std,7654.94288,153661.615648,4.355229,1.852946,960.961095
min,0.00000,27.000000,0.600000,0.000000,1.000000
25%,6629.00000,157851.000000,0.600000,5.000000,2.000000
50%,13258.00000,309581.000000,1.374000,6.000000,5.000000
75%,19887.00000,419542.000000,3.694000,7.000000,28.000000
max,26516.00000,608444.000000,80.773000,10.000000,22186.000000


In [ ]:
tn_movie_budgets_raw.describe()

,id
count,5782.000000
mean,50.372363
std,28.821076
min,1.000000
25%,25.000000
50%,50.000000
75%,75.000000
max,100.000000


In [ ]:
# Describle the non-numerical data
bom_raw.describe(include=object)

,title,studio,foreign_gross
count,3387,3382,2037
unique,3386,257,1204
top,Bluebeard,IFC,1200000
freq,2,166,23


In [ ]:
# Loop through each table and display its head using pandas
for table_name in imdb_tables:
    df = pd.read_sql_query(f"SELECT * from {table_name[0]}", conn)
    print(f"Table: {table_name[0]}")
    print(df.describe(include=object))

Table: movie_basics
         movie_id primary_title original_title       genres
count      146144        146144         146123       140736
unique     146144        136071         137773         1085
top     tt0063540          Home         Broken  Documentary
freq            1            24             19        32185
Table: directors
         movie_id  person_id
count      291174     291174
unique     140417     109253
top     tt4050462  nm6935209
freq         3818        238
Table: known_for
        person_id   movie_id
count     1638260    1638260
unique     576444     514781
top     nm1202937  tt0806910
freq            6        633
Table: movie_akas
         movie_id       title  region language        types attributes
count      331703      331703  278410    41715       168447      14925
unique     122302      252781     213       76           10         77
top     tt2488496  Robin Hood      US       en  imdbDisplay  new title
freq           61          32   51490    22895       1

In [ ]:
# Create a copy for cleaning
# NEED THIS STEP???

## Handling Column Names (NEEDED?)
We will start by cleaning the column names so that they will be easy to access.
We will:
- Strip spaces
- Replace periods with underscores
- lowercase all column names

## Handling Duplicates
We will check for duplicates and see how we should handle them

In [ ]:
#Checks bom_raw for duplicates
bom_raw.duplicated().value_counts()

False    3387
dtype: int64

In [ ]:
# Loop through each table and display its head using pandas
for table_name in imdb_tables:
    df = pd.read_sql_query(f"SELECT * from {table_name[0]}", conn)
    print(f"Table: {table_name[0]}")
    print(df.duplicated().value_counts(),"\n")

Table: movie_basics
False    146144
dtype: int64 

Table: directors
False    163535
True     127639
dtype: int64 

Table: known_for
False    1638260
dtype: int64 

Table: movie_akas
False    331703
dtype: int64 

Table: movie_ratings
False    73856
dtype: int64 

Table: persons
False    606648
dtype: int64 

Table: principals
False    1028186
dtype: int64 

Table: writers
False    178352
True      77521
dtype: int64 



In [ ]:
# Loop through each table and display its head using pandas
df = pd.read_sql_query(f"SELECT movie_id,person_id, COUNT(*) as 'Duplicates' from directors GROUP BY movie_id,person_id HAVING COUNT(*) > 1", conn)
df

,movie_id,person_id,Duplicates
0,tt0063540,nm0712540,4
1,tt0069049,nm0000080,2
2,tt0100275,nm0749914,2
3,tt0100275,nm0765384,2
4,tt0146592,nm1030585,2
...,...,...,...
54672,tt9916538,nm8185151,3
54673,tt9916622,nm9272490,2
54674,tt9916622,nm9272491,2
54675,tt9916754,nm8349149,2


Duplicates appear to be due to directors being attributed to a movie more than once, so we drop them below

We remove duplicates from the datasets here

In [ ]:
# Get a list of all tables in the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Loop through each table and display its head using pandas
for table_name in tables:
    df = pd.read_sql_query(f"SELECT DISTINCT * from {table_name[0]}", conn)
    print(f"Table: {table_name[0]}")
    print(df.duplicated().value_counts(),"\n")

Table: movie_basics
False    146144
dtype: int64 

Table: directors
False    163535
dtype: int64 

Table: known_for
False    1638260
dtype: int64 

Table: movie_akas
False    331703
dtype: int64 

Table: movie_ratings
False    73856
dtype: int64 

Table: persons
False    606648
dtype: int64 

Table: principals
False    1028186
dtype: int64 

Table: writers
False    178352
dtype: int64 



**Observations**
- None so far

## Removing Undesired Data
We don't want all the data so let's remove what we don't want
- Undesired Columns in each dataset
- Undesired values

The dates are in the `Event.Date` column in the format 'YYYY-MM-DD'

In [ ]:
# Check chaging the Event.Date column to dates
pd.to_datetime(aviation_data_cleaned['event_date']).head()

0   1948-10-24
1   1962-07-19
2   1974-08-30
3   1977-06-19
4   1979-08-02
Name: event_date, dtype: datetime64[ns]

In [ ]:
# Update the column to the correct datetime type
aviation_data_cleaned['event_date'] = pd.to_datetime(aviation_data_cleaned['event_date'])

We also want to remove columns with low data coverage

In [ ]:
missing_data = (aviation_data_cleaned.isna().sum() / len(aviation_data_cleaned)).sort_values(ascending=False)
missing_data

schedule                  0.881817
air_carrier               0.827179
aircraft_category         0.661247
far_description           0.660470
longitude                 0.615430
latitude                  0.615331
airport_code              0.408621
airport_name              0.377583
broad_phase_of_flight     0.251609
total_serious_injuries    0.134753
total_fatal_injuries      0.127188
total_minor_injuries      0.126807
total_uninjured           0.056882
engine_type               0.036854
purpose_of_flight         0.026098
number_of_engines         0.021779
aircraft_damage           0.015540
weather_condition         0.007594
injury_severity           0.000706
location                  0.000141
model                     0.000085
make                      0.000000
event_date                0.000000
dtype: float64

**Observations**
- We can likely exclude the columns with significant missing data
- We should keep those relevant to our business questions such as `airport_code` and `airport_name`

In [ ]:
# Pull out columns to drop
columns_to_drop = missing_data[missing_data.values > 0.6].index.to_list()
columns_to_drop

['schedule',
 'air_carrier',
 'aircraft_category',
 'far_description',
 'longitude',
 'latitude']

In [ ]:
aviation_data_cleaned = aviation_data_cleaned.drop(columns_to_drop, axis=1)
aviation_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70848 entries, 7 to 90347
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_date              70848 non-null  datetime64[ns]
 1   location                70838 non-null  object        
 2   airport_code            41898 non-null  object        
 3   airport_name            44097 non-null  object        
 4   injury_severity         70798 non-null  object        
 5   aircraft_damage         69747 non-null  object        
 6   make                    70848 non-null  object        
 7   model                   70842 non-null  object        
 8   number_of_engines       69305 non-null  float64       
 9   engine_type             68237 non-null  object        
 10  purpose_of_flight       68999 non-null  object        
 11  total_fatal_injuries    61837 non-null  float64       
 12  total_serious_injuries  61301 non-null  float6

## Handling NaNs
- Here we look through each dataset and handle Null values


### bom Database

In [124]:
# Check out the data again
bom_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [ ]:
# Explore the number of nulls
bom_raw.isna().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

### Imdb Database

In [128]:
# Loop through each table and display its head using pandas
for table_name in imdb_tables:
    df = pd.read_sql_query(f"SELECT * from {table_name[0]}", conn)
    print(f"Table: {table_name[0]}")
    print(df.info(),"\n")

Table: movie_basics
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB
None 

Table: directors
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291174 entries, 0 to 291173
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   291174 non-null  object
 1   person_id  291174 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB
None 

Table: known_for
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1638260 entries, 0 to 16

In [129]:
# Explore the number of nulls
bom_raw.isna().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

There are only 10 events with missing location data which is a crucial piece of of information so we can drop them

In [ ]:
# Check data
aviation_data_cleaned = aviation_data_cleaned[aviation_data_cleaned['location'].notna()]

In [ ]:
# Check to confirm we dropped nulles
aviation_data_cleaned['location'].isna().sum()

0

## Creating New Columns

The first column we will create will be the passenger count collumn. We will be assuming that:
- Passenger Count = [Total Injured] + [Total Uninjured]

In [ ]:
aviation_data_cleaned['passenger_count'] = aviation_data_cleaned[['total_fatal_injuries','total_serious_injuries','total_minor_injuries','total_uninjured']].sum(axis=1)

In [ ]:
aviation_data_cleaned

,event_date,location,airport_code,airport_name,injury_severity,aircraft_damage,make,model,number_of_engines,engine_type,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,passenger_count
7,1982-01-01,"PULLMAN, WA",Unknown,BLACKBURN AG STRIP,Non-Fatal,Substantial,Cessna,140,1.0,Reciprocating,Personal,0.0,0.0,0.0,2.0,VMC,Takeoff,2.0
8,1982-01-01,"EAST HANOVER, NJ",N58,HANOVER,Non-Fatal,Substantial,Cessna,401B,2.0,Reciprocating,Business,0.0,0.0,0.0,2.0,IMC,Landing,2.0
9,1982-01-01,"JACKSONVILLE, FL",JAX,JACKSONVILLE INTL,Non-Fatal,Substantial,North American,NAVION L-17B,1.0,Reciprocating,Personal,0.0,0.0,3.0,0.0,IMC,Cruise,3.0
10,1982-01-01,"HOBBS, NM",Unknown,Unknown,Non-Fatal,Substantial,Piper,PA-28-161,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Approach,1.0
11,1982-01-01,"TUSKEGEE, AL",Unknown,TUSKEGEE,Non-Fatal,Substantial,Beech,V35B,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Landing,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90340,2022-12-21,"Auburn Hills, MI",Unknown,Unknown,Minor,Unknown,CESSNA,172F,1.0,Reciprocating,Personal,0.0,1.0,0.0,0.0,Unknown,Unknown,1.0
90341,2022-12-21,"Reserve, LA",Unknown,Unknown,Minor,Unknown,GRUMMAN AMERICAN AVN. CORP.,AA-5B,1.0,Reciprocating,Instructional,0.0,1.0,0.0,1.0,Unknown,Unknown,2.0
90343,2022-12-26,"Annapolis, MD",Unknown,Unknown,Minor,Unknown,PIPER,PA-28-151,1.0,Reciprocating,Personal,0.0,1.0,0.0,0.0,Unknown,Unknown,1.0
90345,2022-12-26,"Payson, AZ",PAN,PAYSON,Non-Fatal,Substantial,AMERICAN CHAMPION AIRCRAFT,8GCBC,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Unknown,1.0


## Column Cleaning

This section is for cleaning columns.
- Clean make to make sure there aren't duplicate makes
- Separate out City and State

### Makes

In [ ]:
# Start by checking out the different makes
aviation_data_cleaned['make'].unique()

array(['Cessna', 'North American', 'Piper', ..., 'PHANTOM',
       'JAMES R DERNOVSEK', 'ORLICAN S R O'], dtype=object)

In [ ]:
# Find the current number of unique makes
len(aviation_data_cleaned['make'].unique())

2050

In [ ]:
# Change names to be capitalized and check the length
cleaned_makes = aviation_data_cleaned['make'].str.title().unique()
print(cleaned_makes)
len(cleaned_makes)

['Cessna' 'North American' 'Piper' ... 'Phantom' 'James R Dernovsek'
 'Orlican S R O']


1683

In [ ]:
# That looks like we cleaned a bunch of duplicates, let's assign the column and check
aviation_data_cleaned['make'] = aviation_data_cleaned['make'].str.title()

In [ ]:
# Check the length
len(aviation_data_cleaned['make'].unique())

1683

### Clean the Top 5 makes specifically

In [ ]:
aviation_data_cleaned.shape

(70245, 18)

In [ ]:
## Create a list of top brands and a map
top_makes_list = ['Beech', 'Cessna', 'Cirrus', 'Piper', 'Diamond',
                  'Lockheed', 'Airbus', 'Grumman', 'Raytheon', 'Boeing']
top_makes_map = {}

for make in top_makes_list:
    beech_dict = aviation_data_cleaned[aviation_data_cleaned['make']
                                       .str.contains(make)]['make'].value_counts().to_frame()
    beech_dict['make'] = make
    top_makes_map.update(dict(beech_dict['make']))
# Check out the top makes map
top_makes_map

{'Beech': 'Beech',
 'Beechcraft': 'Beech',
 'Hawker Beechcraft Corp': 'Beech',
 'Hawker Beechcraft': 'Beech',
 'Hawker Beechcraft Corporation': 'Beech',
 'Hawker Beechcraft Corp.': 'Beech',
 'Hawker-Beechcraft': 'Beech',
 'Hawker-Beechcraft Corporation': 'Beech',
 'Beechcraft Corporation': 'Beech',
 'Hawker Beech': 'Beech',
 'Cessna': 'Cessna',
 'Cessna Aircraft Co': 'Cessna',
 'Cessna Ector': 'Cessna',
 'Cessna/Air Repair Inc': 'Cessna',
 'Cessna Wren': 'Cessna',
 'Cessna Aircraft Co.': 'Cessna',
 'Cessna Robertson': 'Cessna',
 'Cessna Soloy': 'Cessna',
 'Cessna/Weaver': 'Cessna',
 'Cessna Aircraft': 'Cessna',
 'Cirrus Design Corp': 'Cirrus',
 'Cirrus': 'Cirrus',
 'Cirrus Design Corp.': 'Cirrus',
 'Cirrus Design Corporation': 'Cirrus',
 'Cirrus Design': 'Cirrus',
 'Piper': 'Piper',
 'Piper Aircraft Inc': 'Piper',
 'Piper/Cub Crafters': 'Piper',
 'New Piper Aircraft Inc': 'Piper',
 'Piper Aircraft': 'Piper',
 'New Piper': 'Piper',
 'Piper Aerostar': 'Piper',
 "Piper/Wally'S Flyers Inc"

In [ ]:
# Apply the Top Makes Map to the data
top_makes_filter = aviation_data_cleaned['make'].isin(top_makes_list)
aviation_data_cleaned[top_makes_filter]['make'] = aviation_data_cleaned['make'].map(top_makes_map)

<ipython-input-96-9edcf83bf972>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aviation_data_cleaned[top_makes_filter]['make'] = aviation_data_cleaned['make'].map(top_makes_map)


In [ ]:
aviation_data_cleaned.shape

(70245, 18)

In [ ]:
aviation_data_cleaned['top_make'] = aviation_data_cleaned['make'].isin(top_makes_list)

In [ ]:
aviation_data_cleaned['top_make'].value_counts(normalize=True)

True     0.656516
False    0.343484
Name: top_make, dtype: float64

In [ ]:
len(aviation_data_cleaned[aviation_data_cleaned['top_make']]['make'].value_counts())

10

In [ ]:
aviation_data_cleaned[aviation_data_cleaned['top_make']].groupby('make').count()

,event_date,location,airport_code,airport_name,injury_severity,aircraft_damage,model,number_of_engines,engine_type,purpose_of_flight,...,weather_condition,broad_phase_of_flight,passenger_count,top_make,use_category,top_model,city,state,fatality_rate,percent_uninjured
make,,,,,,,,,,,,,,,,,,,,,
Airbus,69,69,69,69,69,69,69,69,69,69,...,69,69,69,69,69,69,69,69,69,69
Beech,4838,4838,4838,4838,4838,4838,4838,4838,4838,4838,...,4838,4838,4838,4838,4838,4838,4838,4838,4838,4838
Boeing,868,868,868,868,868,868,868,868,868,868,...,868,868,868,868,868,868,868,868,868,868
Cessna,25157,25157,25157,25157,25157,25157,25157,25157,25157,25157,...,25157,25157,25157,25157,25157,25157,25157,25157,25157,25157
Cirrus,96,96,96,96,96,96,96,96,96,96,...,96,96,96,96,96,96,96,96,96,96
Diamond,35,35,35,35,35,35,35,35,35,35,...,35,35,35,35,35,35,35,35,35,35
Grumman,1110,1110,1110,1110,1110,1110,1110,1110,1110,1110,...,1110,1110,1110,1110,1110,1110,1110,1110,1110,1110
Lockheed,66,66,66,66,66,66,66,66,66,66,...,66,66,66,66,66,66,66,66,66,66
Piper,13830,13830,13830,13830,13830,13830,13830,13830,13830,13830,...,13830,13830,13830,13830,13830,13830,13830,13830,13830,13830


### Determine Commercial or Private

In [ ]:
def private_or_commercial(make):
    private = ['Beech', 'Cessna', 'Cirrus', 'Piper', 'Diamond']
    commercial = ['Lockheed', 'Airbus', 'Grumman', 'Raytheon', 'Boeing']
    if make in private:
        return 'Private Enterprise'
    elif make in commercial:
        return 'Commercial'
    else:
        return 'Unknown'

In [ ]:
# Create a column to designate commercial or private
aviation_data_cleaned['use_category'] = aviation_data_cleaned['make'].apply(private_or_commercial)
aviation_data_cleaned['use_category'].value_counts()

Private Enterprise    43986
Unknown               24128
Commercial             2131
Name: use_category, dtype: int64

### Top 3 Models per make

In [ ]:
# Remove exccess white space from data and remove "-"
aviation_data_cleaned['model'] = aviation_data_cleaned['model'].str.strip().str.replace('-','').str.replace(' ','')
aviation_data_cleaned['model'].head()

7            140
8           401B
9     NAVIONL17B
10       PA28161
11          V35B
Name: model, dtype: object

In [ ]:
def model_cleaning_function(model):
    model = str(model)
    if model.startswith('1') or model.startswith('2'):
         model = model[:3]
    elif any(map(model.startswith, ['PA','A','DA','G','SR','C','L'])):
        model = model[:4]
    elif model.startswith('HAWKER'):
         model = 'HAWKER'
    else: 
        model = model
    return model

In [ ]:
aviation_data_cleaned['model'] = aviation_data_cleaned['model'].apply(model_cleaning_function)

In [ ]:
top_commercial_models_list = ['A330', '747', '777', '727', 'CRJ', 'A320', '747', '737', 'DC3']

In [ ]:
only_private = aviation_data_cleaned['use_category'] == 'Private Enterprise'
top_models_list = aviation_data_cleaned[top_makes_filter & only_private].groupby(['make','model']).count().reset_index()
top_models_list = list(top_models_list.sort_values('event_date',ascending=False).groupby('make').head(3)['model'])
top_models_list.extend(top_commercial_models_list)
top_models_list

['172',
 'PA28',
 '150',
 '182',
 'PA18',
 'PA32',
 'A36',
 'C23',
 '58',
 'SR22',
 'DA20',
 'SR20',
 'DA40',
 'DA42',
 'A330',
 '747',
 '777',
 '727',
 'CRJ',
 'A320',
 '747',
 '737',
 'DC3']

**Top Commercial Models**
- Airbus A330
- Boeing 747
- Boeing 777
- Boeing 727
- CRJ Series
- Airbus A320
- Boeing 747
- Embraer E-Jet Family
- Boeing 737
- Douglas DC-3

In [ ]:
def commercial_model_cleaning_function(model):
    top_commercial_models_list = ['A330', '747', '777', '727', 'CRJ', 'A320', '747', '737', 'DC3']
    model = str(model)
    for top_model in top_commercial_models_list:
        if model.startswith(top_model):
            return top_model
            break
        else:
            return model

In [ ]:
aviation_data_cleaned['model'] = aviation_data_cleaned['model'].apply(commercial_model_cleaning_function)

In [ ]:
aviation_data_cleaned[aviation_data_cleaned['model'].isin(top_commercial_models_list)]['model'].count()

108

In [ ]:
aviation_data_cleaned['top_model'] = aviation_data_cleaned['model'].isin(top_models_list)

In [ ]:
aviation_data_cleaned['top_model'].value_counts(normalize=True)

False    0.701274
True     0.298726
Name: top_model, dtype: float64

### Location to City & State

In [ ]:
# Add new city and state columns
aviation_data_cleaned[['city', 'state']] = aviation_data_cleaned['location'].str.rsplit(', ', 1, expand = True)

In [ ]:
aviation_data_cleaned['city'] = aviation_data_cleaned['city'].str.capitalize()

In [ ]:
aviation_data_cleaned[['city', 'state']].head()

,city,state
7,Pullman,WA
8,East hanover,NJ
9,Jacksonville,FL
10,Hobbs,NM
11,Tuskegee,AL


In [ ]:
# Check for missing data
aviation_data_cleaned[['city', 'state']].isna().sum()

city      0
state    51
dtype: int64

In [ ]:
# There are only a few so we can remove them
aviation_data_cleaned = aviation_data_cleaned[aviation_data_cleaned['state'].notna()]

In [ ]:
# Confirm the missing data is gone
aviation_data_cleaned[['city', 'state']].isna().sum()

city     0
state    0
dtype: int64

In [ ]:
# Confirm the missing data is gone
aviation_data_cleaned.isna().sum()

event_date                0
location                  0
airport_code              0
airport_name              0
injury_severity           0
aircraft_damage           0
make                      0
model                     0
number_of_engines         0
engine_type               0
purpose_of_flight         0
total_fatal_injuries      0
total_serious_injuries    0
total_minor_injuries      0
total_uninjured           0
weather_condition         0
broad_phase_of_flight     0
passenger_count           0
top_make                  0
use_category              0
top_model                 0
city                      0
state                     0
dtype: int64

### Weather conditions

In [ ]:
aviation_data_cleaned['weather_condition'].value_counts()

VMC        63918
IMC         5231
Unknown      484
UNK          473
Unk           88
Name: weather_condition, dtype: int64

In [ ]:
# Update weather conditions names
weather_map = { 'VMC' : 'Visual Meteorological Conditions',
                 'IMC' : 'Instrument Meteorological Conditions',
                 'Unknown' : 'Unknown Meteorological Conditions',
                 'UNK' : 'Unknown Meteorological Conditions',
                 'Unk' : 'Unknown Meteorological Conditions'
}

aviation_data_cleaned['weather_condition'] = aviation_data_cleaned['weather_condition'].map(weather_map)

<ipython-input-121-41ea5d073c37>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aviation_data_cleaned['weather_condition'] = aviation_data_cleaned['weather_condition'].map(weather_map)


### Injury Severity

In [ ]:
# Check out injury severity
aviation_data_cleaned['injury_severity'].value_counts()

Non-Fatal      57453
Fatal(1)        4556
Fatal(2)        2989
Fatal           2851
Fatal(3)         955
Fatal(4)         663
Minor            178
Fatal(5)         166
Serious          128
Fatal(6)         104
Fatal(7)          30
Fatal(8)          26
Fatal(10)         15
Unavailable       14
Fatal(9)           8
Fatal(14)          5
Fatal(12)          5
Fatal(13)          3
Fatal(11)          3
Fatal(18)          3
Fatal(23)          2
Fatal(20)          2
Fatal(17)          2
Fatal(25)          2
Fatal(228)         1
Fatal(21)          1
Fatal(15)          1
Fatal(19)          1
Fatal(88)          1
Fatal(49)          1
Fatal(78)          1
Fatal(153)         1
Fatal(44)          1
Fatal(31)          1
Fatal(16)          1
Fatal(92)          1
Fatal(37)          1
Fatal(28)          1
Fatal(135)         1
Fatal(64)          1
Fatal(110)         1
Fatal(230)         1
Fatal(73)          1
Fatal(111)         1
Fatal(68)          1
Fatal(132)         1
Fatal(265)         1
Fatal(34)    

In [ ]:
# Clean up the injury severity
aviation_data_cleaned['injury_severity'] = aviation_data_cleaned['injury_severity'].str.split('(', 1, expand = True)

<ipython-input-123-8a01635113c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aviation_data_cleaned['injury_severity'] = aviation_data_cleaned['injury_severity'].str.split('(', 1, expand = True)


In [ ]:
aviation_data_cleaned['injury_severity'].head()

7     Non-Fatal
8     Non-Fatal
9     Non-Fatal
10    Non-Fatal
11    Non-Fatal
Name: injury_severity, dtype: object

## Create key metrics
- Fatality rate
- % uninjured

In [ ]:
# Create a column call fatality rate to figure out the deadliness of the accident
aviation_data_cleaned['fatality_rate'] = aviation_data_cleaned['total_fatal_injuries']/aviation_data_cleaned['passenger_count']
# Create a column call fatality rate to figure out the deadliness of the accident
aviation_data_cleaned['percent_uninjured'] = aviation_data_cleaned['total_uninjured']/aviation_data_cleaned['passenger_count']

<ipython-input-125-f7f133f62319>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aviation_data_cleaned['fatality_rate'] = aviation_data_cleaned['total_fatal_injuries']/aviation_data_cleaned['passenger_count']
<ipython-input-125-f7f133f62319>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aviation_data_cleaned['percent_uninjured'] = aviation_data_cleaned['total_uninjured']/aviation_data_cleaned['passenger_count']


In [ ]:
aviation_data_cleaned.fillna(0, inplace=True)

/Users/willbennett/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


## Conclusion
We now have a cleaned up data set with mostly non-null values and have focused on the columns we need

In [132]:
bom_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
